In [20]:
import osmnx as ox
import geopandas as gpd
import numpy as np
import pandas as pd
import numpy as np

##### Koramangala, Indira Nagar and surrounding areas do not have any road network data available so I have used Newyork city data

In [4]:
# Taking NewYork city
# (took help from google regarding the code and api as well)
place_name = "New York City, New York, USA"
graph = ox.graph_from_place(place_name, network_type='drive')

In [15]:
# Fetching road network data from OpenStreetMap 
graph = ox.graph_from_place(place_name, network_type='drive')
edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)

In [17]:
edges.columns

Index(['osmid', 'oneway', 'highway', 'reversed', 'length', 'geometry', 'lanes',
       'ref', 'name', 'maxspeed', 'bridge', 'access', 'tunnel', 'width',
       'junction', 'est_width'],
      dtype='object')

#### Columns that are most relevant to estimate travel time and understanding road characteristics

* Length
* Max speed
* Lanes
* Highway

In [23]:
# Selecting relevant columns and adding additional features
edges_relevant = edges[['length', 'maxspeed', 'lanes', 'highway']]

# additional features
edges_relevant['current_speed'] = np.random.uniform(20, 60, len(edges_relevant))  # Current speed in km/h
edges_relevant['traffic_condition'] = np.random.choice([0.8, 1.0, 1.2], len(edges_relevant))  # Traffic multipliers

C:\Users\a\AppData\Local\Temp\ipykernel_8908\2384129723.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges_relevant['current_speed'] = np.random.uniform(20, 60, len(edges_relevant))  # Current speed in km/h
C:\Users\a\AppData\Local\Temp\ipykernel_8908\2384129723.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges_relevant['traffic_condition'] = np.random.choice([0.8, 1.0, 1.2], len(edges_relevant))  # Traffic multipliers


In [24]:
# Calculating the initial ETA for each segment (took help from google)
edges_relevant['ETA_minutes'] = (edges_relevant['length'] / (edges_relevant['current_speed'] * edges_relevant['traffic_condition'])) * 60

C:\Users\a\AppData\Local\Temp\ipykernel_8908\2785971566.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges_relevant['ETA_minutes'] = (edges_relevant['length'] / (edges_relevant['current_speed'] * edges_relevant['traffic_condition'])) * 60


In [25]:
# Function to update ETA dynamically based on new current speed and traffic conditions
def update_eta(edges, current_speeds, traffic_conditions):
    edges['current_speed'] = current_speeds
    edges['traffic_condition'] = traffic_conditions
    edges['ETA_minutes'] = (edges['length'] / (edges['current_speed'] * edges['traffic_condition'])) * 60
    return edges

In [26]:
# Simulating a dynamic environment where the ETA is updated as the vehicle moves
# Example: Updating current speed and traffic condition for the first 5 segments

new_speeds = np.random.uniform(20, 60, 5)
new_traffic_conditions = np.random.choice([0.8, 1.0, 1.2], 5)

In [27]:
# Updating the ETA for the first 5 segments dynamically
edges_relevant.iloc[:5] = update_eta(edges_relevant.iloc[:5], new_speeds, new_traffic_conditions)

# Save the updated synthetic data
edges_relevant.to_csv('dynamic_synthetic.csv', index=False)

C:\Users\a\AppData\Local\Temp\ipykernel_8908\336187900.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges['current_speed'] = current_speeds
C:\Users\a\AppData\Local\Temp\ipykernel_8908\336187900.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges['traffic_condition'] = traffic_conditions
C:\Users\a\AppData\Local\Temp\ipykernel_8908\336187900.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



### Moving forward, this dynamic data can be used to perform model training and evaluation.